<a href="https://colab.research.google.com/github/Tuevu110405/Machine-translation-Chinese-to-Vietnamese-/blob/main/Training_and_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training


## Install dataset and needed packages

In [ ]:
!gdown 1KuDLpUaSe0wzDWEhgG0wKXV1_AVmzLK4
!unzip ./dataset.zip

Downloading...
From: https://drive.google.com/uc?id=1KuDLpUaSe0wzDWEhgG0wKXV1_AVmzLK4
To: /content/dataset.zip
100% 928k/928k [00:00<00:00, 141MB/s]
Archive:  ./dataset.zip
   creating: dataset/
   creating: dataset/private_test/
  inflating: dataset/private_test/private_test.zh  
   creating: dataset/public_test/
  inflating: dataset/public_test/public_test.zh  
   creating: dataset/train/
  inflating: dataset/train/train.vi  
  inflating: dataset/train/train.zh  


In [ ]:
import os


folder_id = '1DOyc8HUdz5z4XhuCG0xtR3wKc8bbS6_5'
output_dir = os.getcwd()
os.makedirs(output_dir, exist_ok=True)
!gdown --folder $folder_id -O $output_dir

Retrieving folder contents
Processing file 1ypF2QKRMdX3r82mJ4eAEP1gL_gpQbEe0 checkpoint_epoch_40.pt
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1ypF2QKRMdX3r82mJ4eAEP1gL_gpQbEe0
From (redirected): https://drive.google.com/uc?id=1ypF2QKRMdX3r82mJ4eAEP1gL_gpQbEe0&confirm=t&uuid=f02067a7-698f-4605-ac54-e419e8d810a6
To: /content/checkpoint_epoch_40.pt
100% 1.89G/1.89G [00:14<00:00, 130MB/s]
Download completed


In [ ]:
import os
import shutil

output_dir = "./checkpoint_bidirectional"
file_to_move = "./checkpoint_epoch_40.pt"

os.makedirs(output_dir, exist_ok=True)
shutil.move(file_to_move, os.path.join(output_dir, os.path.basename(file_to_move)))
print(f"Moved {file_to_move} to {output_dir}")


Moved ./checkpoint_epoch_40.pt to ./checkpoint_bidirectional


In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.4 MB/s eta 0:00:00


In [ ]:
import os
import random
import sentencepiece as spm
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import math
from dataclasses import dataclass
from typing import List, Tuple, Any, Dict, Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import sacrebleu


## Preprocessing

In [ ]:
BASE_DIR = "."
TRAIN_DIR = os.path.join(BASE_DIR, "dataset", "train")
SRC_FILE = os.path.join(TRAIN_DIR, "train.zh")
TGT_FILE = os.path.join(TRAIN_DIR, "train.vi")
TOKENIZER_DIR = os.path.join(BASE_DIR, "tokenizer_train32")
TOKENIZER_PREFIX = os.path.join(TOKENIZER_DIR, "spm_zh_vi_joint")
MAX_TOKENS = 32
VOCAB_SIZE = 8000
USER_SYMBOLS = ["<2zh>", "<2vi>"]

os.makedirs(TOKENIZER_DIR, exist_ok = True)

def load_lines(path):
    with open(path, "r", encoding = "utf-8") as f:
        return [l.strip() for l in f if l.strip()]

zh_lines = load_lines(SRC_FILE)
vi_lines = load_lines(TGT_FILE)

In [ ]:
import os
import sentencepiece as spm
import numpy as np
from sklearn.model_selection import train_test_split


# Giả định các biến cấu hình đã được định nghĩa từ trước
# TOKENIZER_DIR = "./tokenizer"
# TOKENIZER_PREFIX = "m_model"
# VOCAB_SIZE = 32000
# USER_SYMBOLS = ["<2vi>", "<2zh>"] # Ví dụ các token đặc biệt

# Train sentencepiece
temp_corpus = os.path.join(TOKENIZER_DIR, "temp_corpus.txt")

# Gộp dữ liệu tiếng Trung và Tiếng Việt vào chung 1 file để train Shared Vocabulary
with open(temp_corpus, "w", encoding="utf-8") as fout:
    for zh, vi in zip(zh_lines, vi_lines):
        fout.write(zh + "\n")
        fout.write(vi + "\n")

# Configuration for SentencePiece
# The spm_args string construction is no longer needed with keyword arguments.

# Training
spm.SentencePieceTrainer.Train(
    input=temp_corpus,
    model_prefix=TOKENIZER_PREFIX,
    vocab_size=VOCAB_SIZE,
    model_type="bpe",
    character_coverage=1.0,
    pad_id=0,
    unk_id=1,
    bos_id=2,
    eos_id=3,
    user_defined_symbols=USER_SYMBOLS # Pass as a list directly
)

print(f"Tokenizer saved to {TOKENIZER_PREFIX}.model")

#LOAD TOKENIZER
sp = spm.SentencePieceProcessor()
sp.Load(f"{TOKENIZER_PREFIX}.model")

# 3. RELOAD RAW DATA (Tải lại dữ liệu gốc) -----
def load_lines(path):
    with open(path, "r", encoding="utf-8") as f:
        return [l.strip() for l in f if l.strip()]

full_zh_lines = load_lines(SRC_FILE)
full_vi_lines = load_lines(TGT_FILE)
assert len(zh_lines) == len(vi_lines)

# --- 2. CHIA TẬP TRAIN / VAL / TEST (8:1:1) ---
# Bước 1: Tách 80% cho Train, còn lại 20% cho Temp (Val + Test)
train_zh, temp_zh, train_vi, temp_vi = train_test_split(
    full_zh_lines, full_vi_lines, test_size=0.2, random_state=42, shuffle=True
)

# Bước 2: Tách 20% Temp thành 10% Val và 10% Test (Chia đôi Temp)
val_zh, test_zh, val_vi, test_vi = train_test_split(
    temp_zh, temp_vi, test_size=0.5, random_state=42, shuffle=True
)

print(f"Train size: {len(train_zh)}")
print(f"Val size:   {len(val_zh)}")
print(f"Test size:  {len(test_zh)}")

#
MAX_TOK = 64  # Ngưỡng độ dài tối đa (token)

def tok_len(t, prefix=None):
    """Hàm tính độ dài token của một câu"""
    if prefix:
        t = f"{prefix} {t}" # Thêm token định hướng dịch (vd: <2vi>)
    return len(sp.encode(t, out_type=int))

# Tính độ dài trước khi lọc (chỉ để thống kê)
zh_before = [tok_len(z, "<2vi>") for z in zh_lines]
vi_before = [tok_len(v) for v in vi_lines]

# Lọc bỏ các cặp câu quá dài
filtered_zh, filtered_vi = [], []
for z, v in zip(zh_lines, vi_lines):
    # Chỉ giữ lại nếu CẢ nguồn và đích đều ngắn hơn MAX_TOK
    if tok_len(z, "<2vi>") <= MAX_TOK and tok_len(v) <= MAX_TOK:
        filtered_zh.append(z)
        filtered_vi.append(v)

# Chuyển sang numpy array để tiện xử lý thống kê sau này
zh_after = np.array([tok_len(z, "<2vi>") for z in filtered_zh])
vi_after = np.array([tok_len(v) for v in filtered_vi])

OUT_DIR = "./clean_data"
os.makedirs(OUT_DIR, exist_ok=True)

zh_out = os.path.join(OUT_DIR, f"train_maxlen{MAX_TOK}.zh")
vi_out = os.path.join(OUT_DIR, f"train_maxlen{MAX_TOK}.vi")

with open(zh_out, "w", encoding="utf-8") as fzh:
    fzh.write("\n".join(filtered_zh))

with open(vi_out, "w", encoding="utf-8") as fvi:
    fvi.write("\n".join(filtered_vi))

Tokenizer saved to ./tokenizer_train32/spm_zh_vi_joint.model
Train size: 25648
Val size:   3206
Test size:  3207


In [ ]:
import os
import torch
import random
import sentencepiece as spm
from dataclasses import dataclass

@dataclass
class RopeConfig:
    train_src_file: str
    train_tgt_file: str
    spm_prefix: str

    vocab_size: int = 8000
    d_model: int = 768
    n_heads: int = 12
    n_kv_heads: int = 4
    num_encoder_layers: int = 8
    num_decoder_layers: int = 8
    d_ff: int = 3072
    dropout: float = 0.01
    max_len: int = 32
    rope_base: float = 10000.0

    pad_token: str = "<pad>"
    bos_token: str = "<s>"
    eos_token: str = "</s>"
    unk_token: str = "<unk>"
    zh_token: str = "<2zh>"
    vi_token: str = "<2vi>"

    #Training argument
    batch_size  = 128
    num_epochs = 40
    lr_base = 2e-4
    warmup_steps = 200
    label_smoothing = 0.01
    weight_decay: float = 0.01
    grad_clip = 1.0
    span_mask_prob = 0.01
    vi2zh_epoch_ratio = 0.7
    save_every: int = 3

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    num_workers = 8
    save_dir = "./checkpoint_bidirectional"
    seed = 42



config = RopeConfig(
    train_src_file=os.path.join(OUT_DIR, f"train_maxlen{MAX_TOK}.zh"),
    train_tgt_file=os.path.join(OUT_DIR, f"train_maxlen{MAX_TOK}.vi"),
    spm_prefix=TOKENIZER_PREFIX,
)

random.seed(config.seed)
torch.manual_seed(config.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(config.seed)

os.makedirs(config.save_dir, exist_ok=True)

## Build Dataset, DataLoader

In [ ]:
import math
import random
import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from typing import List, Tuple, Dict, Any
from torch.nn.utils.rnn import pad_sequence

class BidirectionalTranslationDataset(Dataset):
    """
    Dataset sinh dữ liệu hai chiều (Zh->Vi và Vi->Zh)
    kết hợp kỹ thuật Span Masking (tạo nhiễu).
    """

    def __init__(
        self,
        src_lines: List[str],
        tgt_lines: List[str],
        sp_model, # sentencepiece model
        config,   # RopeConfig
        is_training: bool = True,
    ):
        self.sp = sp_model
        self.config = config
        self.is_training = is_training

        # Lấy ID của các token đặc biệt
        self.pad_id = sp_model.piece_to_id(config.pad_token)
        self.bos_id = sp_model.piece_to_id(config.bos_token)
        self.eos_id = sp_model.piece_to_id(config.eos_token)
        self.unk_id = sp_model.piece_to_id(config.unk_token)
        self.zh_id = sp_model.piece_to_id(config.zh_token)
        self.vi_id = sp_model.piece_to_id(config.vi_token)

        self.samples = []

        if is_training:
            # Luân phiên tạo dữ liệu cho 2 hướng: Zh->Vi và Vi->Zh
            for i, (src, tgt) in enumerate(zip(src_lines, tgt_lines)):
                if i % 2 == 0:
                    # Hướng xuôi: Input = Zh, Label = Vi
                    # (CE_src_text, CE_tgt_text, CL_zh_text, CL_vi_text, direction)
                    self.samples.append(
                        (self.add_lang_token(src, config.vi_token), tgt, src, tgt, "zh2vi")
                    )
                else:
                    # Hướng ngược: Input = Vi, Label = Zh
                    # (CE_src_text, CE_tgt_text, CL_zh_text, CL_vi_text, direction)
                    self.samples.append(
                        (self.add_lang_token(tgt, config.zh_token), src, src, tgt, "vi2zh")
                    )
        else:
            # Validation: Chỉ giữ hướng chính Zh->Vi
            for src, tgt in zip(src_lines, tgt_lines):
                self.samples.append(
                    (self.add_lang_token(src, config.vi_token), tgt, src, tgt, "zh2vi")
                )

        # Lưu lại index của từng loại để dùng cho sampler sau này
        self.zh2vi_indices = [
            idx for idx, (_, _, _, _, d) in enumerate(self.samples) if d == "zh2vi"
        ]
        self.vi2zh_indices = [
            idx for idx, (_, _, _, _, d) in enumerate(self.samples) if d == "vi2zh"
        ]

    def add_lang_token(self, text: str, lang_tok: str) -> str:
        """Thêm token ngôn ngữ vào đầu câu (vd: <2vi> Yêu em)"""
        text = text.strip()
        if text.startswith("<2vi>") or text.startswith("<2zh>"):
            return text
        return f"{lang_tok} {text}"

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, str]:
        # Unpack all 5 items stored in self.samples
        src_text_ce, tgt_text_ce, cl_zh_text, cl_vi_text, direction = self.samples[idx]

        # Encode for Cross-Entropy Loss (main translation task)
        src_ids_ce = self.sp.encode(src_text_ce, out_type=int)[: self.config.max_len]
        tgt_ids_ce = [self.bos_id] + self.sp.encode(tgt_text_ce, out_type=int) + [self.eos_id]
        tgt_ids_ce = tgt_ids_ce[: self.config.max_len]

        # Apply span masking for CE source if training and direction is zh2vi
        # Span masking should only apply to the actual source of the CE task
        if self.is_training and direction == "zh2vi":
            src_ids_ce = self.apply_span_masking(src_ids_ce)

        # Encode for Contrastive Learning (original sentences without special tokens/masking)
        # Ensure BOS/EOS/PAD for CL are handled consistently with model embedding layer expectations
        cl_zh_ids = self.sp.encode(cl_zh_text, out_type=int)[:self.config.max_len] # No BOS/EOS here, as mean_pool expects clean sentences
        cl_vi_ids = self.sp.encode(cl_vi_text, out_type=int)[:self.config.max_len]

        return (
            torch.tensor(src_ids_ce, dtype=torch.long),
            torch.tensor(tgt_ids_ce, dtype=torch.long),
            torch.tensor(cl_zh_ids, dtype=torch.long),
            torch.tensor(cl_vi_ids, dtype=torch.long),
            direction
        )

    def apply_span_masking(self, src_ids: List[int]) -> List[int]:
        """
        Thay thế ngẫu nhiên một đoạn token bằng <unk> với xác suất nhỏ.
        Giúp mô hình học cách 'đoán' từ dựa trên ngữ cảnh (giống BERT).
        """
        if not self.is_training or random.random() > self.config.span_mask_prob:
            return src_ids

        src_ids = src_ids.copy()
        # Không mask các token đặc biệt
        special = {self.pad_id, self.bos_id, self.eos_id, self.zh_id, self.vi_id}
        maskable = [i for i, t in enumerate(src_ids) if t not in special]

        if not maskable:
            return src_ids

        # Chọn độ dài đoạn mask (1 hoặc 2 token)
        num_to_mask = min(random.randint(1, 2), len(maskable))
        start = random.choice(maskable)

        for i in range(start, min(start + num_to_mask, len(src_ids))):
            # Mask với xác suất 70% trong vùng đã chọn
            if i in maskable and random.random() < 0.7:
                src_ids[i] = self.unk_id
        return src_ids


def select_vi2zh_window(indices: List[int], epoch: int, ratio: float) -> List[int]:
    """
    Chọn một cửa sổ trượt (sliding window) các mẫu Vi->Zh.
    Mỗi epoch sẽ chọn một phần khác nhau của dữ liệu dịch ngược.
    """
    if not indices or ratio <= 0:
        return []

    total = len(indices)
    # Tính kích thước cửa sổ dựa trên ratio (tỷ lệ)
    window = max(1, int(math.ceil(total * min(ratio, 1.0))))

    # Tính vị trí bắt đầu và kết thúc dựa trên số epoch (xoay vòng)
    start = ((epoch - 1) * window) % total
    end = start + window

    if end <= total:
        return indices[start:end]

    # Xử lý trường hợp cửa sổ trượt qua cuối danh sách (nối phần cuối với phần đầu)
    wrap = end - total
    return indices[start:] + indices[:wrap]

def collate_fn(batch): # This is the training collate function
    """
    Batch đầu vào là list các tuple: (src_tensor_ce, tgt_tensor_ce, cl_zh_tensor, cl_vi_tensor, direction_string)
    """
    # 1. Tách các thành phần từ batch (now expects 5 items from __getitem__)
    src_ids_ce_list, tgt_ids_ce_list, cl_zh_ids_list, cl_vi_ids_list, directions = zip(*batch)

    # 2. Tạo batch tổng cho Cross Entropy Loss (Padding bình thường)
    PAD_ID = 0

    src_batch_ce = pad_sequence(src_ids_ce_list, batch_first=True, padding_value=PAD_ID)
    tgt_batch_ce = pad_sequence(tgt_ids_ce_list, batch_first=True, padding_value=PAD_ID)

    # 3. Phân loại dữ liệu cho Contrastive Learning
    zh_vi_src_cl = [] # Chinese source for Zh->Vi contrastive pairs
    vi_vi_tgt_cl = [] # Vietnamese target for Zh->Vi contrastive pairs

    vi_zh_src_cl = [] # Vietnamese source for Vi->Zh contrastive pairs
    zh_zh_tgt_cl = [] # Chinese target for Vi->Zh contrastive pairs

    for i in range(len(batch)):
        direction = directions[i]
        if direction == "zh2vi":
            zh_vi_src_cl.append(cl_zh_ids_list[i])
            vi_vi_tgt_cl.append(cl_vi_ids_list[i])
        else: # vi2zh
            vi_zh_src_cl.append(cl_vi_ids_list[i])
            zh_zh_tgt_cl.append(cl_zh_ids_list[i])

    # Helper để pad danh sách con (tránh lỗi nếu danh sách rỗng)
    def safe_pad(tensor_list):
        if not tensor_list:
            # Trả về tensor rỗng kích thước [0, 1] để không lỗi model forward
            return torch.zeros(0, 1, dtype=torch.long)
        return pad_sequence(tensor_list, batch_first=True, padding_value=PAD_ID)

    # 4. Đóng gói vào Dictionary
    return {
        'src': src_batch_ce,          # Dùng cho CE Loss (Tổng hợp)
        'tgt': tgt_batch_ce,          # Dùng cho CE Loss (Tổng hợp)

        # Dùng cho Contrastive Loss (Phân tách)
        'ids_zh_vi': safe_pad(zh_vi_src_cl),
        'ids_vi_vi': safe_pad(vi_vi_tgt_cl),

        'ids_vi_zh': safe_pad(vi_zh_src_cl),
        'ids_zh_zh': safe_pad(zh_zh_tgt_cl),
    }

def collate_fn_eval(batch): # This is the evaluation collate function
    """
    Collate function for evaluation, expects 5 items from __getitem__ but only uses 2.
    """
    # Unpack all 5 items returned by __getitem__
    srcs_ce, tgts_ce, _, _, _ = zip(*batch) # Ignore CL-specific items and direction

    PAD_ID = 0

    max_src = max(len(s) for s in srcs_ce)
    max_tgt = max(len(t) for t in tgts_ce)

    src_pad = torch.zeros(len(batch), max_src, dtype = torch.long)
    tgt_pad = torch.zeros(len(batch), max_tgt, dtype = torch.long)

    for i, (s, t) in enumerate(zip(srcs_ce, tgts_ce)):
        src_pad[i, :len(s)] = s
        tgt_pad[i, :len(t)] = t

    return src_pad, tgt_pad

def build_train_loader(
    train_dataset: BidirectionalTranslationDataset,
    epoch: int,
    config,
) -> Tuple[DataLoader, int]:
    """
    Tạo DataLoader kết hợp:
    1. Toàn bộ dữ liệu chính (Zh->Vi)
    2. Một phần dữ liệu phụ (Vi->Zh) thay đổi theo epoch
    """
    # Lấy toàn bộ index của hướng xuôi
    active = list(train_dataset.zh2vi_indices)

    # Lấy một phần index của hướng ngược
    vi_slice = select_vi2zh_window(
        train_dataset.vi2zh_indices, epoch, config.vi2zh_epoch_ratio
    )
    active.extend(vi_slice)

    # Dùng SubsetRandomSampler để chỉ lấy mẫu trong danh sách 'active'
    sampler = SubsetRandomSampler(active)

    loader = DataLoader(
        train_dataset,
        batch_size=config.batch_size,
        sampler=sampler,
        collate_fn=collate_fn, # Use the updated training collate_fn
        num_workers=config.num_workers,
        pin_memory=True,
    )
    return loader, len(vi_slice)


In [ ]:
train_dataset = BidirectionalTranslationDataset(
    train_zh, train_vi, sp, config, is_training=True
)

# Tập Val & Test: is_training=False (Chỉ giữ chiều xuôi Zh->Vi, không nhiễu)
val_dataset = BidirectionalTranslationDataset(
    val_zh, val_vi, sp, config, is_training=False
)

test_dataset = BidirectionalTranslationDataset(
    test_zh, test_vi, sp, config, is_training=False
)

# B. Khởi tạo DataLoader
# Train Loader: Cần dùng hàm đặc biệt để xử lý sliding window
# (Sẽ gọi lại trong vòng lặp huấn luyện từng epoch)
# train_loader, _ = build_train_loader(train_dataset, epoch=1, config=config)

# Val & Test Loader: Dùng DataLoader tiêu chuẩn (không cần sampler phức tạp)
val_loader = DataLoader(
    val_dataset,
    batch_size=config.batch_size,
    shuffle=False,
    collate_fn=collate_fn
)

test_loader = DataLoader(
    test_dataset,
    batch_size=config.batch_size,
    shuffle=False,
    collate_fn=collate_fn
)

## Model

In [ ]:
class RMSNorm(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-8):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        rms = torch.sqrt(torch.mean(x * x, dim=-1, keepdim=True) + self.eps)
        return self.weight * x / rms

class RoPE(nn.Module):
    def __init__(self, d_model: int, base: float = 10000.0):
        super().__init__()
        inv_freq = 1.0 / (base ** (torch.arange(0, d_model, 2).float() / d_model))
        self.register_buffer("inv_freq", inv_freq)
        self._cos = None
        self._sin = None
        self._seq_len_cached = 0

    def _maybe_update_cache(self, seq_len, device, dtype):
        if seq_len > self._seq_len_cached or self._cos is None or self._cos.device != device:
            self._seq_len_cached = seq_len
            positions = torch.arange(seq_len, device = device, dtype = dtype)
            freqs = torch.outer(positions, self.inv_freq.to(device))
            self._cos = torch.cos(freqs)
            self._sin = torch.sin(freqs)

    def forward(self, x, seq_len):
        seq_len = seq_len or x.size(-2)
        self._maybe_update_cache(seq_len, x.device, x.dtype)
        # Corrected: RoPE should return cos and sin components, not multiply x
        return self._cos[:seq_len], self._sin[:seq_len]

def apply_rope(x: torch.Tensor, cos: torch.Tensor, sin: torch.Tensor) -> torch.Tensor:
    # x shape: (..., seq_len, head_dim)
    head_dim = x.shape[-1]
    x1 = x[..., 0::2]
    x2 = x[..., 1::2]

    # Reshape cos, sin để broadcast đúng: (1, 1, seq_len, head_dim/2)
    # Lưu ý: cos, sin từ hàm RoPE của bạn đang có shape (seq_len, head_dim/2)
    cos = cos.unsqueeze(0).unsqueeze(0)
    sin = sin.unsqueeze(0).unsqueeze(0)

    rot1 = x1 * cos - x2 * sin
    rot2 = x1 * sin + x2 * cos
    return torch.cat([rot1, rot2], dim=-1)


class FFN_SwiGLU(nn.Module):
    def __init__(self, d_model, d_ff, dropout):
        super().__init__()
        self.d_ff = d_ff  # <--- Thêm dòng này
        self.linear1 = nn.Linear(d_model, 2 * d_ff, bias=False)
        self.linear2 = nn.Linear(d_ff, d_model, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        h = self.linear1(x)
        g, v = h[..., :self.d_ff], h[..., self.d_ff:]
        # Công thức đúng: (SiLU(g) * v)
        # F.silu(g) tương đương g * sigmoid(g)
        s = F.silu(g) * v  # <--- Sửa logic nhân với v
        out = self.linear2(s)
        return self.dropout(out)

class GroupedQueryAttentionRoPE(nn.Module):
    def __init__(self, d_model, n_heads, n_kv_heads, dropout, rope_base):
        super().__init__()
        assert n_heads % n_kv_heads == 0
        self.n_heads = n_heads
        self.n_kv_heads = n_kv_heads
        self.d_k = d_model // n_heads
        # Corrected: Define n_groups
        self.n_groups = self.n_heads // self.n_kv_heads
        self.W_q = nn.Linear(d_model, n_heads * self.d_k, bias = False)
        self.W_k = nn.Linear(d_model, n_kv_heads * self.d_k, bias = False)
        self.W_v = nn.Linear(d_model, n_kv_heads * self.d_k, bias = False)
        self.W_o = nn.Linear(d_model, d_model, bias = False)
        self.dropout = nn.Dropout(dropout)
        self.scale = math.sqrt(self.d_k)
        self.rope = RoPE(self.d_k, base = rope_base)

    def forward(self, q, k, v, key_padding_mask = None, attn_mask = None):
        B, T_q = q.size(0), q.size(1)
        T_k = k.size(1)

        Q = self.W_q(q).view(B, T_q, self.n_heads, self.d_k).transpose(1,2)
        K = self.W_k(k).view(B, T_k, self.n_kv_heads, self.d_k).transpose(1,2)
        V = self.W_v(v).view(B, T_k, self.n_kv_heads, self.d_k).transpose(1,2)

        cos_q, sin_q = self.rope(Q, T_q)
        cos_k, sin_k = self.rope(K, T_k)

        Q = apply_rope(Q, cos_q, sin_q)
        K = apply_rope(K, cos_k, sin_k)

        K = K.repeat_interleave(self.n_groups, dim = 1)
        V = V.repeat_interleave(self.n_groups, dim = 1)

        scores = torch.matmul(Q, K.transpose(-2, -1)) / self.scale
        if key_padding_mask is not None:
            scores = scores.masked_fill(key_padding_mask.unsqueeze(1).unsqueeze(2), float("-inf"))
        if attn_mask is not None:
            # Corrected: Remove unsqueeze calls to allow proper broadcasting of (T, T) mask
            scores = scores.masked_fill(attn_mask, float("-inf"))
        attn = F.softmax(scores, dim = -1)
        attn = self.dropout(attn)
        out = torch.matmul(attn, V)
        out = out.transpose(1,2).contiguous().view(B, T_q, -1)
        return self.W_o(out)

class EncoderLayer(nn.Module):
    def __init__(self, config: RopeConfig):
        super().__init__()
        self.ln1 = RMSNorm(config.d_model)
        self.self_attn = GroupedQueryAttentionRoPE(
            config.d_model,
            config.n_heads,
            config.n_kv_heads,
            config.dropout,
            config.rope_base
        )
        self.ln2 = RMSNorm(config.d_model)
        self.ffn = FFN_SwiGLU(config.d_model, config.d_ff, config.dropout)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x, src_pad_mask = None):
        x1 = self.ln1(x)
        attn = self.self_attn(x1, x1, x1, key_padding_mask = src_pad_mask)
        x = x + self.dropout(attn)
        x2 = self.ln2(x)
        return x + self.ffn(x2)

class DecoderLayer(nn.Module):
    def __init__(self, config: RopeConfig):
        super().__init__()
        self.ln1 = RMSNorm(config.d_model)
        self.self_attn = GroupedQueryAttentionRoPE(
            config.d_model,
            config.n_heads,
            config.n_kv_heads,
            config.dropout,
            config.rope_base
        )
        self.ln2 = RMSNorm(config.d_model)
        self.cross_attn = GroupedQueryAttentionRoPE(
            config.d_model, config.n_heads, config.n_kv_heads, config.dropout, config.rope_base
        )
        self.ln3 = RMSNorm(config.d_model)
        self.ffn = FFN_SwiGLU(config.d_model, config.d_ff, config.dropout)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, y, enc_out, tgt_pad_mask = None, tgt_casual_mask = None, src_pad_mask = None):
        y1 = self.ln1(y)
        # Use self attention with casual mask
        self_attn = self.self_attn(y1, y1, y1,
                                   key_padding_mask = tgt_pad_mask,
        attn_mask = tgt_casual_mask)
        y = y + self.dropout(self_attn)
        y2 = self.ln2(y)
        #Cross attention (K, V is the output of the encoder)
        cross_attn = self.cross_attn(y2, enc_out, enc_out,
                                      key_padding_mask = src_pad_mask)
        y = y + self.dropout(cross_attn)
        y3 = self.ln3(y)
        return y + self.ffn(y3)

class TransformerModel(nn.Module):
    def __init__(self, config: RopeConfig, vocab_size: int):
        super().__init__()
        self.config = config
        self.embedding = nn.Embedding(vocab_size, config.d_model, padding_idx = 0
                                    )
        self.emb_dropout = nn.Dropout(config.dropout)
        self.encoder_layers = nn.ModuleList([EncoderLayer(config) for _ in range(config.num_encoder_layers)])
        self.encoder_final_ln = RMSNorm(config.d_model)
        self.decoder_layers = nn.ModuleList([DecoderLayer(config) for _ in range(config.num_decoder_layers)])
        self.decoder_final_ln = RMSNorm(config.d_model)

        self.output_bias = nn.Parameter(torch.zeros(vocab_size))
        self.emb_scale = math.sqrt(config.d_model)
        self._init_weight()

    def _init_weight(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
    def forward(self, src_ids: torch.Tensor, tgt_ids: torch.Tensor):
        src_pad = (src_ids == 0)
        tgt_pad = (tgt_ids == 0)

        tgt_in = tgt_ids[:, : -1]
        tgt_pad_in = tgt_pad[:, : -1]
        T = tgt_in.size(1)
        tgt_causal = torch.triu(
            torch.ones(T, T, dtype = torch.bool, device = tgt_in.device), diagonal = 1
        )

        src_emb = self.emb_dropout(self.embedding(src_ids) * self.emb_scale)
        enc_out = src_emb
        for layer in self.encoder_layers:
            enc_out = layer(enc_out,src_pad)
        enc_out = self.encoder_final_ln(enc_out)

        tgt_emb = self.emb_dropout(self.embedding(tgt_in) * self.emb_scale)
        dec_out = tgt_emb
        for layer in self.decoder_layers:
            dec_out = layer(dec_out, enc_out, tgt_pad_in, tgt_causal, src_pad)
        dec_out = self.decoder_final_ln(dec_out)

        return F.linear(dec_out, self.embedding.weight, self.output_bias)

## Criterion

In [ ]:
class LabelSmoothedCrossEntropyLoss(nn.Module):
    def __init__(self,smoothing: float = 0.1, ignore_index: int = 0):
        super().__init__()
        self.smoothing = smoothing
        self.ignore_index = ignore_index

    def forward(self, logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        vocab_size = logits.size(-1)
        log_probs = F.log_softmax(logits, dim = -1)

        mask = targets != self.ignore_index

        with torch.no_grad():
            true_dist = torch.full_like(
                log_probs, self.smoothing / (vocab_size - 1)
            )
            true_dist.scatter_(1, targets.data.unsqueeze(1), 1 - self.smoothing)
            true_dist[targets == self.ignore_index] = 0.0

        loss = -(true_dist * log_probs).sum(dim=-1)
        loss = loss.masked_fill(~mask,0.0)
        return loss.sum() / mask.sum().clamp(min=1)

## Utils

In [ ]:
class WarmupInverseSqrtScheduler:
    def __init__(self, optimizer, warmup_steps: int, lr_base: float):
        self.optimizer = optimizer
        self.warmup_steps = warmup_steps
        self.lr_base = lr_base
        self.step_num = 0

    def step(self):
        self.step_num += 1
        if self.step_num <= self.warmup_steps:
            lr = self.lr_base * self.step_num / self.warmup_steps
        else:
            lr = self.lr_base * math.sqrt(self.warmup_steps) / math.sqrt(self.step_num)
        for group in self.optimizer.param_groups:
            group["lr"] = lr

    def get_lr(self):
        return self.optimizer.param_groups[0]["lr"]


def train_epoch(model, dataloader, criterion, optimizer, scheduler, config: RopeConfig, epoch: int) -> float:
    model.train()
    total_loss = 0
    pbar = tqdm(dataloader, desc=f"Epoch {epoch}")
    for src_batch, tgt_batch in pbar:
        src_batch = src_batch.to(config.device)
        tgt_batch = tgt_batch.to(config.device)
        logits = model(src_batch, tgt_batch)
        targets = tgt_batch[:, 1:]
        loss = criterion(logits.reshape(-1, logits.size(-1)), targets.reshape(-1))
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_clip)
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()
        pbar.set_postfix(loss=f"{loss.item():.4f}", lr=f"{scheduler.get_lr():.6f}")
    return total_loss / len(dataloader)


@dataclass
class BeamSearchHypothesis:
    tokens: list
    log_prob: float


@torch.no_grad()
def greedy_decode(
    model: TransformerModel,
    src_ids: torch.Tensor,
    sp_model: spm.SentencePieceProcessor,
    config: RopeConfig,
    max_len: int = 32,
):
    """Fast greedy decoding for validation BLEU calculation"""
    model.eval()
    batch_size = src_ids.size(0)
    device = src_ids.device
    bos = sp_model.piece_to_id(config.bos_token)
    eos = sp_model.piece_to_id(config.eos_token)
    pad = sp_model.piece_to_id(config.pad_token)

    # Encode source
    src_pad_mask = (src_ids == pad)
    src_emb = model.embedding(src_ids) * model.emb_scale
    src_input = model.emb_dropout(src_emb)
    enc_out = src_input
    for layer in model.encoder_layers:
        enc_out = layer(enc_out, src_pad_mask)
    enc_out = model.encoder_final_ln(enc_out)

    # Initialize decoder input
    tgt_ids = torch.full((batch_size, 1), bos, dtype=torch.long, device=device)

    for _ in range(max_len):
        tgt_pad = (tgt_ids == pad)
        tgt_len = tgt_ids.size(1)
        tgt_causal = torch.triu(torch.ones(tgt_len, tgt_len, dtype=torch.bool, device=device), diagonal=1)

        tgt_emb = model.embedding(tgt_ids) * model.emb_scale
        tgt_input = model.emb_dropout(tgt_emb)
        dec_out = tgt_input
        for layer in model.decoder_layers:
            dec_out = layer(dec_out, enc_out, tgt_pad, tgt_causal, src_pad_mask)
        dec_out = model.decoder_final_ln(dec_out)

        logits = F.linear(dec_out, model.embedding.weight, model.output_bias)
        next_tokens = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
        tgt_ids = torch.cat([tgt_ids, next_tokens], dim=1)

        # Stop if all sequences have EOS
        if (next_tokens.squeeze(-1) == eos).all():
            break

    # Decode to text
    decoded = []
    for seq in tgt_ids:
        tokens = seq[1:].cpu().tolist()  # Skip BOS
        if eos in tokens:
            tokens = tokens[:tokens.index(eos)]
        decoded.append(sp_model.decode(tokens))

    return decoded


@torch.no_grad()
def beam_search_decode(
    model: TransformerModel,
    src_ids: torch.Tensor,
    sp_model: spm.SentencePieceProcessor,
    config: RopeConfig,
    beam_size: int = 3,
    top_k: int = 3,
    max_len: int = 32,
    length_penalty: float = 0.6,
):
    model.eval()
    batch_size = src_ids.size(0)
    device = src_ids.device
    bos = sp_model.piece_to_id(config.bos_token)
    eos = sp_model.piece_to_id(config.eos_token)
    pad = sp_model.piece_to_id(config.pad_token)
    src_pad_mask = (src_ids == pad)
    src_emb = model.embedding(src_ids) * model.emb_scale
    src_input = model.emb_dropout(src_emb)
    enc_out = src_input
    for layer in model.encoder_layers:
        enc_out = layer(enc_out, src_pad_mask)
    enc_out = model.encoder_final_ln(enc_out)
    decoded = []
    for b in range(batch_size):
        beams = [BeamSearchHypothesis(tokens=[bos], log_prob=0.0)]
        finished = []
        single_enc = enc_out[b : b + 1]
        single_mask = src_pad_mask[b : b + 1]
        for _ in range(max_len):
            new_beams = []
            for beam in beams:
                if beam.tokens[-1] == eos:
                    finished.append(beam)
                    continue
                tgt_ids = torch.tensor(beam.tokens, dtype=torch.long, device=device).unsqueeze(0)
                tgt_pad = (tgt_ids == pad)
                tgt_len = tgt_ids.size(1)
                tgt_causal = torch.triu(torch.ones(tgt_len, tgt_len, dtype=torch.bool, device=device), diagonal=1)
                tgt_emb = model.embedding(tgt_ids) * model.emb_scale
                tgt_input = model.emb_dropout(tgt_emb)
                dec_out = tgt_input
                for layer in model.decoder_layers:
                    dec_out = layer(dec_out, single_enc, tgt_pad, tgt_causal, single_mask)
                dec_out = model.decoder_final_ln(dec_out)
                logits = F.linear(dec_out, model.embedding.weight, model.output_bias)
                log_probs = F.log_softmax(logits[:, -1, :], dim=-1)
                if top_k and top_k > 0:
                    top_val, top_idx = torch.topk(log_probs, min(top_k, log_probs.size(-1)))
                    mask = torch.full_like(log_probs, float("-inf"))
                    mask.scatter_(1, top_idx, top_val)
                    log_probs = mask
                top_vals, top_idx = torch.topk(log_probs.squeeze(0), beam_size)
                for val, idx in zip(top_vals.tolist(), top_idx.tolist()):
                    new_beams.append(BeamSearchHypothesis(tokens=beam.tokens + [idx], log_prob=beam.log_prob + val))
            beams = sorted(new_beams, key=lambda h: h.log_prob, reverse=True)[:beam_size]
            if not beams:
                break
        finished.extend(beams)
        best = max(finished, key=lambda h: h.log_prob / (len(h.tokens) ** length_penalty))
        tokens = best.tokens[1:]
        if eos in tokens:
            tokens = tokens[: tokens.index(eos)]
        decoded.append(sp_model.decode(tokens))
    return decoded


@torch.no_grad()
def evaluate(model, dataloader, criterion, sp_model, config: RopeConfig, calculate_bleu: bool = True, max_bleu_samples: int = 500):
    """
    Fast evaluation with optional BLEU calculation
    Args:
        calculate_bleu: Whether to calculate BLEU score (slower)
        max_bleu_samples: Maximum number of samples for BLEU calculation
    """
    model.eval()
    total_loss = 0.0
    all_predictions, all_references = [], []

    bleu_count = 0
    for i, (src_batch, tgt_batch) in enumerate(tqdm(dataloader, desc="Evaluating", leave=False)):
        src_batch = src_batch.to(config.device)
        tgt_batch = tgt_batch.to(config.device)

        # Always calculate loss
        logits = model(src_batch, tgt_batch)
        targets = tgt_batch[:, 1:]
        loss = criterion(logits.reshape(-1, logits.size(-1)), targets.reshape(-1))
        total_loss += loss.item()

        # Only calculate BLEU on subset for speed
        if calculate_bleu and bleu_count < max_bleu_samples:
            # Use greedy decoding instead of beam search for speed
            preds = greedy_decode(model, src_batch, sp_model, config, max_len=config.max_len)

            eos = sp_model.piece_to_id(config.eos_token)
            for tgt_seq in tgt_batch:
                ref = tgt_seq[1:].cpu().tolist()
                if eos in ref:
                    ref = ref[: ref.index(eos)]
                all_references.append(sp_model.decode(ref))

            all_predictions.extend(preds)
            bleu_count += len(preds)

            # Stop if we have enough samples for BLEU
            if bleu_count >= max_bleu_samples:
                break

    avg_loss = total_loss / len(dataloader)
    bleu_score = 0.0

    if calculate_bleu and sacrebleu is not None and all_predictions:
        try:
            bleu = sacrebleu.corpus_bleu(all_predictions, [all_references], force=True)
            bleu_score = bleu.score
        except Exception as err:
            print(f"BLEU calculation failed: {err}")

    return avg_loss, bleu_score

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ProjectionHead(nn.Module):
    def __init__(self, input_dim, proj_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, input_dim),
            nn.ReLU(),
            nn.Linear(input_dim, proj_dim)
        )

    def forward(self, x):
        return F.normalize(self.net(x), dim = -1)

def encode_context(model, src_ids, pad_id):
    src_pad_mask = (src_ids == pad_id)
    src_emb = model.embedding(src_ids) * model.emb_scale
    src_input = model.emb_dropout(src_emb)

    enc_out = src_input
    for layer in model.encoder_layers:
        enc_out = layer(enc_out, src_pad_mask)
    enc_out = model.encoder_final_ln(enc_out)
    return enc_out

def mean_pool(enc_out, ids, pad_id, special_ids):
    mask = (ids != pad_id)
    for special in special_ids:
        mask = mask & (ids != special)
    mask = mask.float()

    summed = (enc_out * mask.unsqueeze(-1)).sum(dim = 1)
    denom = mask.sum(dim = 1, keepdim=True).clamp(min = 1) # Corrected 'keepsim' to 'keepdim'
    return summed/denom

def contrastive_loss(z_a, z_b, tau):
    sim = torch.matmul(z_a, z_b.transpose(0,1))/tau
    labels = torch.arange(sim.size(0), device = sim.device)
    loss_i = F.cross_entropy(sim, labels)
    loss_j = F.cross_entropy(sim.transpose(0,1), labels)
    return (loss_i + loss_j)/2

def compute_crosslingual_loss(model,
                              projection,
                              ids_zh_vi,
                              ids_vi_vi,
                              ids_vi_zh,
                              ids_zh_zh,
                              cl_config,
                              pad_id,
                              special_ids):
    if ids_zh_vi.size(0) < 2:
        return torch.zeros(1, device = cl_config.device)

    enc_zh_vi = encode_context(model, ids_zh_vi, pad_id)
    enc_vi_vi = encode_context(model, ids_vi_vi, pad_id)
    enc_vi_zh = encode_context(model, ids_vi_zh, pad_id)
    enc_zh_zh = encode_context(model, ids_zh_zh, pad_id)

    z_zh_vi = projection(mean_pool(enc_zh_vi, ids_zh_vi, pad_id, special_ids))
    z_vi_vi = projection(mean_pool(enc_vi_vi, ids_vi_vi, pad_id, special_ids))
    z_vi_zh = projection(mean_pool(enc_vi_zh, ids_vi_zh, pad_id, special_ids))
    z_zh_zh = projection(mean_pool(enc_zh_zh, ids_zh_zh, pad_id, special_ids))

    cl_vi_space = contrastive_loss(z_zh_vi, z_vi_vi, cl_config.contrastive_tau)
    cl_zh_space = contrastive_loss(z_vi_zh, z_zh_zh, cl_config.contrastive_tau)

    return (cl_vi_space + cl_zh_space) * 0.5

def contrastive_train_epoch(model, projection, dataloader,optimizer, criterion, scheduler, cl_config, pad_id, special_ids, global_step, epoch):
    model.train()
    projection.train()

    total_ce_loss = 0.0
    total_cl_loss = 0.0
    total_loss = 0.0
    pbar = tqdm(dataloader, desc = f"Epoch {epoch}")
    for batch in pbar:
        src = batch['src'].to(cl_config.device)
        tgt = batch['tgt'].to(cl_config.device)
        ids_zh_vi = batch['ids_zh_vi'].to(cl_config.device)
        ids_vi_zh = batch['ids_vi_zh'].to(cl_config.device)
        ids_zh_zh = batch['ids_zh_zh'].to(cl_config.device)
        ids_vi_vi = batch['ids_vi_vi'].to(cl_config.device)


        tgt_label = tgt[:, 1:]  # Bỏ token đầu (để dự đoán)
        optimizer.zero_grad()
        logits = model(src, tgt)

        # Tuy nhiên, Label để tính loss thì ta phải tự cắt bên ngoài
        tgt_label = tgt[:, 1:] # Bỏ token đầu <bos> để làm nhãn thực tế

        ce_loss = criterion(
            logits.reshape(-1, logits.size(-1)),
            tgt_label.reshape(-1)
        )

        warmup_ratio = min(1.0, global_step / max(1, cl_config.cross_warmup_steps))
        lambda_cross = cl_config.cross_lambda_max * warmup_ratio

        if ids_zh_vi.size(0) > 0 and ids_vi_zh.size(0) > 0:
             cl_loss = compute_crosslingual_loss(
                model, projection,
                ids_zh_vi, ids_vi_vi, ids_vi_zh, ids_zh_zh, # Lưu ý thứ tự tham số
                cl_config, pad_id, special_ids
            )
        else:
            # Nếu batch thiếu dữ liệu để đối chiếu, bỏ qua CL loss batch này
            cl_loss = torch.tensor(0.0, device=cl_config.device)

        total_batch_loss = ce_loss + lambda_cross * cl_loss
        total_batch_loss.backward()
        torch.nn.utils.clip_grad_norm_(list(model.parameters()) + list(projection.parameters()), config.grad_clip)
        optimizer.step()
        scheduler.step()

        total_ce_loss += ce_loss.item()
        total_cl_loss += cl_loss.item()
        total_loss += total_batch_loss.item()
        global_step += 1

        pbar.set_postfix({
            'ce': f'{ce_loss.item():.4f}',
            'cl': f'{cl_loss.item():.4f}',
            'λ': f'{lambda_cross:.3f}',
            'lr': f'{scheduler.get_lr():.6f}'
        })

    return total_ce_loss / len(dataloader), total_cl_loss / len(dataloader), total_loss / len(dataloader),global_step

# Training stage 1
- Objective function: Cross Entropy loss
- Training for 40 epochs

In [ ]:
from torch.optim import AdamW

In [ ]:
def package_tokenizer(prefix: str) -> Dict[str, Any]:
    model_path = f"{prefix}.model"
    vocab_path = f"{prefix}.vocab"
    if not (os.path.isfile(model_path) and os.path.isfile(vocab_path)):
        raise FileNotFoundError(f"Missing tokenizer files at {prefix}.[model|vocab]")
    with open(model_path, "rb") as f:
        model_bytes = f.read()
    with open(vocab_path, "rb") as f:
        vocab_bytes = f.read()
    return {"prefix": prefix, "model_bytes": model_bytes, "vocab_bytes": vocab_bytes}

In [ ]:
vocab_size = sp.get_piece_size()
print(f"Vocab size: {vocab_size}")
model = TransformerModel(config, vocab_size).to(config.device)
criterion = LabelSmoothedCrossEntropyLoss(config.label_smoothing, ignore_index = 0)
optimizer = AdamW(model.parameters(),
                  lr = config.lr_base,
                  betas = (0.9, 0.98),
                  eps = 1e-9,
                  weight_decay = config.weight_decay
)
scheduler = WarmupInverseSqrtScheduler(optimizer, config.warmup_steps, config.lr_base)
tokenizer = package_tokenizer(config.spm_prefix)
tokenizer_payload = package_tokenizer(config.spm_prefix)

print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")
print(f"checkpoints will be saved to {config.save_dir}")

best_val_loss = float('inf')
best_val_bleu = 0.0

for epoch in range(1, config.num_epochs + 1):
    train_loader, num_vi2zh_samples = build_train_loader(
        train_dataset,
        epoch=epoch,
        config=config
    )
    train_loss = train_epoch(model, train_loader, criterion, optimizer, scheduler, config, epoch)
    val_loss, val_bleu = evaluate(model, val_loader, criterion, sp, config)
    print(f"\nEpoch {epoch}/{config.num_epochs}")
    print(f"  vi→zh coverage: {num_vi2zh_samples}/{len(train_dataset.vi2zh_indices)} (~{100 * config.vi2zh_epoch_ratio:.1f}% per epoch)")
    print(f"  Train Loss: {train_loss:.4f}")
    print(f"  Valid Loss: {val_loss:.4f}")
    print(f"  Valid BLEU: {val_bleu:.2f}")
    print(f"  Learning Rate: {scheduler.get_lr():.6f}")

    if epoch % config.save_every == 0:
        ckpt_path = os.path.join(config.save_dir, f"checkpoint_epoch_{epoch}.pt")
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "train_loss": train_loss,
                "val_loss": val_loss,
                "val_bleu": val_bleu,
                "config": config,
                "tokenizer": tokenizer_payload,
            },
            ckpt_path,
        )
        print(f"  ✓ Saved checkpoint to {ckpt_path}")

    # Update best model based on validation loss (always available)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_val_bleu = val_bleu

        best_path = os.path.join(config.save_dir, "best_model.pt")
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "train_loss": train_loss,
                "val_loss": val_loss,
                "val_bleu": best_val_bleu,
                "config": config,
                "tokenizer": tokenizer_payload,
            },
            best_path,
        )

print("\nTraining finished.")
print(f"Best valid loss: {best_val_loss:.4f} | Best BLEU: {best_val_bleu:.2f}")



Vocab size: 8000
Total parameters: 157179200
checkpoints will be saved to ./checkpoint_bidirectional


Epoch 1: 100%|██████████| 171/171 [01:42<00:00,  1.66it/s, loss=4.6988, lr=0.000171]



Epoch 1/40
  vi→zh coverage: 8977/12824 (~70.0% per epoch)
  Train Loss: 5.9643
  Valid Loss: 0.6850
  Valid BLEU: 4.48
  Learning Rate: 0.000171
  ✓ Saved checkpoint to ./checkpoint_bidirectional/checkpoint_epoch_1.pt


Epoch 2: 100%|██████████| 171/171 [01:45<00:00,  1.63it/s, loss=3.9446, lr=0.000153]



Epoch 2/40
  vi→zh coverage: 8977/12824 (~70.0% per epoch)
  Train Loss: 4.1668
  Valid Loss: 0.5514
  Valid BLEU: 8.64
  Learning Rate: 0.000153
  ✓ Saved checkpoint to ./checkpoint_bidirectional/checkpoint_epoch_2.pt


...


Epoch 38/40
  vi→zh coverage: 8977/12824 (~70.0% per epoch)
  Train Loss: 0.1668
  Valid Loss: 0.5089
  Valid BLEU: 23.92
  Learning Rate: 0.000035
  ✓ Saved checkpoint to ./checkpoint_bidirectional/checkpoint_epoch_38.pt


Epoch 39: 100%|██████████| 171/171 [01:44<00:00,  1.63it/s, loss=0.1650, lr=0.000035]



Epoch 39/40
  vi→zh coverage: 8977/12824 (~70.0% per epoch)
  Train Loss: 0.1658
  Valid Loss: 0.5074
  Valid BLEU: 23.23
  Learning Rate: 0.000035
  ✓ Saved checkpoint to ./checkpoint_bidirectional/checkpoint_epoch_39.pt


Epoch 40: 100%|██████████| 171/171 [01:45<00:00,  1.62it/s, loss=0.1704, lr=0.000034]



Epoch 40/40
  vi→zh coverage: 8977/12824 (~70.0% per epoch)
  Train Loss: 0.1657
  Valid Loss: 0.5055
  Valid BLEU: 23.53
  Learning Rate: 0.000034
  ✓ Saved checkpoint to ./checkpoint_bidirectional/checkpoint_epoch_40.pt

Training finished.
Best valid loss: 0.4449 | Best BLEU: 17.83


# Training stage 2
- Using a small projector and training it to learn latent semantic space of both Chinese and Vietnamese
- Objective: next token prediction as previous stage and teach model semantic feature of a sentence in both Vietnamese and Chinese
- Train first 20 epochs apply Early Stopper(patience= 5)s to minimize loss function of Valid dataset(Stop at 10 epochs)
- Train  next 20 epochs  apply Early Stopper(patience= 10)s to aximize sacrebleu score of Valid dataset(Stop at 12 epochs)
- Totally, I trained my model for 22 epochs


In [ ]:
import os
import torch
import copy

class EarlyStopper:
    def __init__(self, patience=5, min_delta=0, mode='min', save_path=None):
        """
        mode: 'min' (giảm loss) hoặc 'max' (tăng BLEU)
        """
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.save_path = save_path
        self.counter = 0
        self.early_stop = False

        # Khởi tạo giá trị tốt nhất tùy theo mode
        self.best_score = float('inf') if mode == 'min' else float('-inf')

    def __call__(self, score, model, projection, optimizer, epoch, config, tokenizer):
        if self.mode == 'min':
            improved = score < self.best_score - self.min_delta
        else:
            improved = score > self.best_score + self.min_delta

        if improved:
            self.best_score = score
            self.counter = 0
            print(f" -> Score improved to {score:.4f}. Saving checkpoint...")

            # Lưu checkpoint đầy đủ
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'projection_state_dict': projection.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'score': score,
                'config': config,
                'tokenizer': tokenizer
            }, self.save_path)
        else:
            self.counter += 1
            print(f" -> EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True

In [ ]:
def beam_evaluate(
        model: TransformerModel,
        dataloader: DataLoader,
        criterion,
        sp_model: spm.SentencePieceProcessor,
        config: RopeConfig,
        beam_size: int = 3,
        max_bleu_samples:int = 400
):
    model.eval()
    total_loss = 0.0
    all_predictions = []
    all_references = []

    bleu_count = 0

    pbar = tqdm(dataloader, desc = 'Validationg (Beam Search)', leave = False)

    for src_batch, tgt_batch in pbar:
        src_batch = src_batch.to(config.device)
        tgt_batch = tgt_batch.to(config.device)

        logits = model(src_batch, tgt_batch)
        targets = tgt_batch[:, 1:]
        loss = criterion(logits.reshape(-1,logits.size(-1)), targets.reshape(-1))
        total_loss += loss.item()

        if bleu_count < max_bleu_samples:
            preds = beam_search_decode(
                model = model,
                src_ids=src_batch,
                sp_model = sp_model,
                config = config,
                beam_size = beam_size,
                top_k = 3,
                max_len = config.max_len,
                length_penalty = 0.6
            )

            eos = sp_model.piece_to_id(config.eos_token)
            for tgt_seq in tgt_batch:
                ref_ids = tgt_seq[1:].cpu().tolist()
                if eos in ref_ids:
                    ref_ids = ref_ids[:ref_ids.index(eos)]

                ref_text = sp_model.decode(ref_ids)
                all_references.append(ref_text)

            all_predictions.extend(preds)
            bleu_count += len(preds)

            if bleu_count >= max_bleu_samples:
                pbar.set_description('Validation (Beam Search)')
    avg_loss = total_loss / len(dataloader)
    bleu_score = 0.0
    if all_predictions:
        try:
            #Scarebleu require references having type of list of lists
            bleu = sacrebleu.corpus_bleu(all_predictions, [all_references], force = True)
            bleu_score = bleu.score

            # In thử 1 câu mẫu để debug
            print(f"\nExample Pred: {all_predictions[0]}")
            print(f"Example Ref : {all_references[0]}")
        except Exception as e:
            print(f"Lỗi tính BLEU: {e}")

    return avg_loss, bleu_score


In [ ]:
from torch.optim import AdamW

In [ ]:
# ===== CONTRASTIVE LEARNING SETUP =====

# Contrastive training config (define first to avoid memory issues)
@dataclass
class ContrastiveConfig:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    proj_dim: int = 768
    contrastive_tau: float = 0.07
    cross_lambda_max: float = 0.1
    cross_warmup_steps: int = 200
    lr_base: float = 5e-5
    warmup_steps: int = 200
    num_epochs: int = 20
    batch_size: int = 64
    save_dir: str = "./checkpoints_contrastive"
    save_every: int = 5

cl_config = ContrastiveConfig()
os.makedirs(cl_config.save_dir, exist_ok=True)

# Load checkpoint epoch 40 for contrastive training (load to CPU first)
print("Loading checkpoint for contrastive training...")
checkpoint_path = os.path.join(config.save_dir, "checkpoint_epoch_40.pt")

if not os.path.exists(checkpoint_path):
    raise FileNotFoundError(f"Required checkpoint not found: {checkpoint_path}")

print(f"Loading checkpoint: {checkpoint_path}")
# Load to CPU first to avoid GPU memory issues
checkpoint = torch.load(checkpoint_path, map_location=config.device, weights_only=False)

# Update config with smaller batch size for contrastive learning
old_config = checkpoint["config"]
config.batch_size = cl_config.batch_size  # Use smaller batch size
config.num_workers = 2  # Reduce workers

# Load model state dict
# model.load_state_dict(checkpoint["model_state_dict"])
# print(f"Loaded checkpoint from epoch {checkpoint['epoch']}")
# print(f"Updated batch_size from {old_config.batch_size} to {config.batch_size}")

# print(f"Contrastive config: lr={cl_config.lr_base}, epochs={cl_config.num_epochs}")
# print(f"Save dir: {cl_config.save_dir}")

Loading checkpoint for contrastive training...
Loading checkpoint: ./checkpoint_bidirectional/checkpoint_epoch_40.pt


In [ ]:
projection = ProjectionHead(config.d_model, cl_config.proj_dim).to(config.device)
print(f"Projection head: {config.d_model} -> {cl_config.proj_dim}")

Projection head: 768 -> 768


In [ ]:
def package_tokenizer(prefix: str) -> Dict[str, Any]:
    model_path = f"{prefix}.model"
    vocab_path = f"{prefix}.vocab"
    if not (os.path.isfile(model_path) and os.path.isfile(vocab_path)):
        raise FileNotFoundError(f"Missing tokenizer files at {prefix}.[model|vocab]")
    with open(model_path, "rb") as f:
        model_bytes = f.read()
    with open(vocab_path, "rb") as f:
        vocab_bytes = f.read()
    return {"prefix": prefix, "model_bytes": model_bytes, "vocab_bytes": vocab_bytes}

In [ ]:
# --- 1. Khởi tạo Model & Optimizer ---
vocab_size = sp.get_piece_size()
print(f"Vocab size: {vocab_size}")
model = TransformerModel(config, vocab_size ).to(config.device)
model.load_state_dict(checkpoint["model_state_dict"])
projection = ProjectionHead(config.d_model, cl_config.proj_dim).to(config.device)


optimizer = AdamW(
    list(model.parameters()) + list(projection.parameters()),
    lr=cl_config.lr_base,
    betas=(0.9, 0.98),
    eps=1e-9,
    weight_decay=0.01
)

tokenizer_payload = package_tokenizer(config.spm_prefix)


# warmup_steps thường là 4000 hoặc 10% tổng số bước train
scheduler = WarmupInverseSqrtScheduler(
    optimizer=optimizer,
    warmup_steps=4000,
    lr_base=0.0005
)

# --- 3. Cấu hình Early Stopping ---
early_stopper = EarlyStopper(
    patience=5,
    min_delta=0.0001,
    mode='min',
    save_path=os.path.join(cl_config.save_dir, "best_cl_model.pt")
)

criterion = LabelSmoothedCrossEntropyLoss(config.label_smoothing, ignore_index = 0)

special_ids = [
    sp.piece_to_id(config.pad_token),
    sp.piece_to_id(config.bos_token),
    sp.piece_to_id(config.eos_token),
    sp.piece_to_id(config.zh_token),
    sp.piece_to_id(config.vi_token)
]
print(f"Special token IDs: {special_ids}")

print(f"Total parameters: {sum(p.numel() for p in model.parameters())+ sum(j.numel() for j in projection.parameters())}")
print("Start_Training")

global_step = 0 # Initialize global_step here

# Re-initialize val_loader and test_loader to ensure correct collate_fn
val_loader = DataLoader(
    val_dataset,
    batch_size=config.batch_size,
    shuffle=False,
    collate_fn=collate_fn_eval
)

test_loader = DataLoader(
    test_dataset,
    batch_size=config.batch_size,
    shuffle=False,
    collate_fn=collate_fn_eval
)

for epoch in range(1, cl_config.num_epochs + 1):
    train_loader, num_vi2zh_samples = build_train_loader(
        train_dataset,
        epoch=epoch,
        config=config
    )

    print(f"\n{'='*10} EPOCH {epoch} {'='*10}")
    # Hàm train_epoch của bạn đã có dòng `scheduler.step()` bên trong vòng lặp batch
    avg_ce_loss, avg_cl_loss, avg_total_loss, global_step = contrastive_train_epoch(
        model = model,
        projection = projection,
        dataloader = train_loader,
        optimizer = optimizer,
        criterion = criterion,
        scheduler = scheduler,
        cl_config = cl_config,
        pad_id = sp.piece_to_id(config.pad_token),
        special_ids = special_ids,
        global_step = global_step,
        epoch = epoch
    )

    print(f"\nEpoch {epoch}/{cl_config.num_epochs}")
    print(f" - Train CE Loss: {avg_ce_loss:.4f}")
    print(f" - Train CL Loss: {avg_cl_loss:.4f}")
    print(f" - Total Loss:    {avg_total_loss:.4f}")
    print(f" - Learning Rate: {scheduler.get_lr():.8f}")

    # Validate
    val_loss, val_bleu = evaluate(model, val_loader, criterion, sp, config)

    print(f"Val Loss: {val_loss:.4f} | Val BLEU: {val_bleu:.2f}")

    if epoch % cl_config.save_every == 0:
        save_path = os.path.join(cl_config.save_dir, f"cl_checkpoint_epoch_{epoch}.pt")
        torch.save({
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "projection_state_dict": projection.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "loss": avg_total_loss,
            "config": config,
            "tokenizer": tokenizer_payload
        }, save_path)
        print(f" -> Saved periodic checkpoint: {save_path}")

    # Check Early Stopping
    early_stopper(
        score = val_loss,
        model = model,
        projection = projection,
        optimizer = optimizer,
        epoch = epoch,
        config = config,
        tokenizer = tokenizer_payload
    )

    if early_stopper.early_stop:
        print(f"Early stopping triggered at epoch {epoch}")
        break

print("Contrastive Training finished.")

Vocab size: 8000
Special token IDs: [0, 2, 3, 4, 5]
Total parameters: 158360384
Start_Training

========== EPOCH 1 ==========


Epoch 1: 100%|██████████| 341/341 [02:55<00:00,  1.95it/s, ce=0.2048, cl=0.0996, λ=0.100, lr=0.000043]



Epoch 1/20
 - Train CE Loss: 0.1651
 - Train CL Loss: 1.1635
 - Total Loss:    0.2069
 - Learning Rate: 0.00004263


Val Loss: 0.5177 | Val BLEU: 23.59
 -> Score improved to 0.5177. Saving checkpoint...

========== EPOCH 2 ==========


Epoch 2: 100%|██████████| 341/341 [02:57<00:00,  1.92it/s, ce=0.2300, cl=0.0375, λ=0.100, lr=0.000085]


...


Epoch 11/20
 - Train CE Loss: 0.8121
 - Train CL Loss: 0.0949
 - Total Loss:    0.8216
 - Learning Rate: 0.00046887


Val Loss: 0.5007 | Val BLEU: 17.61
 -> EarlyStopping counter: 4 out of 5

========== EPOCH 12 ==========


Epoch 12: 100%|██████████| 341/341 [02:58<00:00,  1.91it/s, ce=0.9787, cl=0.1168, λ=0.100, lr=0.000494]



Epoch 12/20
 - Train CE Loss: 0.8276
 - Train CL Loss: 0.0946
 - Total Loss:    0.8371
 - Learning Rate: 0.00049435


Val Loss: 0.5000 | Val BLEU: 18.42
 -> EarlyStopping counter: 5 out of 5
Early stopping triggered at epoch 12
Contrastive Training finished.


## Stage 2.1: Continue to train for 20 epochs applied early stopper to maximize sacrebleu. Stop at epoch 12

In [ ]:
@dataclass
class ContrastiveConfig:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    proj_dim: int = 768
    contrastive_tau: float = 0.07
    cross_lambda_max: float = 0.1
    cross_warmup_steps: int = 200
    lr_base: float = 5e-5
    warmup_steps: int = 200
    num_epochs: int = 20
    batch_size: int = 64
    save_dir: str = "./checkpoints_contrastive"
    save_every: int = 5

cl_config = ContrastiveConfig()
os.makedirs(cl_config.save_dir, exist_ok=True)

In [ ]:
# --- 1. Khởi tạo Model & Optimizer ---
vocab_size = sp.get_piece_size()
print(f"Vocab size: {vocab_size}")
model = TransformerModel(config, vocab_size ).to(config.device)
checkpoint_path = '/content/checkpoints_contrastive/cl_checkpoint_epoch_10.pt'
checkpoint = torch.load(checkpoint_path, map_location=config.device, weights_only=False)
model.load_state_dict(checkpoint["model_state_dict"])
projection = ProjectionHead(config.d_model, cl_config.proj_dim).to(config.device)
projection.load_state_dict(checkpoint["projection_state_dict"])


optimizer = AdamW(
    list(model.parameters()) + list(projection.parameters()),
    lr=cl_config.lr_base,
    betas=(0.9, 0.98),
    eps=1e-9,
    weight_decay=0.01
)

tokenizer_payload = package_tokenizer(config.spm_prefix)


# warmup_steps thường là 4000 hoặc 10% tổng số bước train
scheduler = WarmupInverseSqrtScheduler(
    optimizer=optimizer,
    warmup_steps=4000,
    lr_base=0.0005
)

# --- 3. Cấu hình Early Stopping ---
early_stopper = EarlyStopper(
    patience=10,
    min_delta=0.0001,
    mode='max',
    save_path=os.path.join(cl_config.save_dir, "best_cl_model.pt")
)

criterion = LabelSmoothedCrossEntropyLoss(config.label_smoothing, ignore_index = 0)

special_ids = [
    sp.piece_to_id(config.pad_token),
    sp.piece_to_id(config.bos_token),
    sp.piece_to_id(config.eos_token),
    sp.piece_to_id(config.zh_token),
    sp.piece_to_id(config.vi_token)
]
print(f"Special token IDs: {special_ids}")

print(f"Total parameters: {sum(p.numel() for p in model.parameters())+ sum(j.numel() for j in projection.parameters())}")
print("Start_Training")

global_step = 0 # Initialize global_step here

# Re-initialize val_loader and test_loader to ensure correct collate_fn
val_loader = DataLoader(
    val_dataset,
    batch_size=config.batch_size,
    shuffle=False,
    collate_fn=collate_fn_eval
)

test_loader = DataLoader(
    test_dataset,
    batch_size=config.batch_size,
    shuffle=False,
    collate_fn=collate_fn_eval
)

for epoch in range(1, cl_config.num_epochs + 1):
    train_loader, num_vi2zh_samples = build_train_loader(
        train_dataset,
        epoch=epoch,
        config=config
    )

    print(f"\n{'='*10} EPOCH {epoch + 10} {'='*10}")
    # Hàm train_epoch của bạn đã có dòng `scheduler.step()` bên trong vòng lặp batch
    avg_ce_loss, avg_cl_loss, avg_total_loss, global_step = contrastive_train_epoch(
        model = model,
        projection = projection,
        dataloader = train_loader,
        optimizer = optimizer,
        criterion = criterion,
        scheduler = scheduler,
        cl_config = cl_config,
        pad_id = sp.piece_to_id(config.pad_token),
        special_ids = special_ids,
        global_step = global_step,
        epoch = epoch
    )

    print(f"\nEpoch {epoch}/{cl_config.num_epochs}")
    print(f" - Train CE Loss: {avg_ce_loss:.4f}")
    print(f" - Train CL Loss: {avg_cl_loss:.4f}")
    print(f" - Total Loss:    {avg_total_loss:.4f}")
    print(f" - Learning Rate: {scheduler.get_lr():.8f}")

    # Validate
    val_loss, val_bleu = evaluate(model, val_loader, criterion, sp, config)

    print(f"Val Loss: {val_loss:.4f} | Val BLEU: {val_bleu:.2f}")

    if epoch % cl_config.save_every == 0:
        save_path = os.path.join(cl_config.save_dir, f"cl_checkpoint_epoch_{epoch}.pt")
        torch.save({
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "projection_state_dict": projection.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "loss": avg_total_loss,
            "config": config,
            "tokenizer": tokenizer_payload
        }, save_path)
        print(f" -> Saved periodic checkpoint: {save_path}")

    # Check Early Stopping
    early_stopper(
        score = val_bleu,
        model = model,
        projection = projection,
        optimizer = optimizer,
        epoch = epoch,
        config = config,
        tokenizer = tokenizer_payload
    )

    if early_stopper.early_stop:
        print(f"Early stopping triggered at epoch {epoch}")
        break

print("Contrastive Training finished.")

Vocab size: 8000
Special token IDs: [0, 2, 3, 4, 5]
Total parameters: 158360384
Start_Training

========== EPOCH 11 ==========


Epoch 1: 100%|██████████| 341/341 [02:55<00:00,  1.94it/s, ce=0.4030, cl=0.0123, λ=0.100, lr=0.000043]



Epoch 1/20
 - Train CE Loss: 0.4800
 - Train CL Loss: 0.0617
 - Total Loss:    0.4839
 - Learning Rate: 0.00004263


Val Loss: 0.4681 | Val BLEU: 22.78
 -> Score improved to 22.7814. Saving checkpoint...

========== EPOCH 12 ==========


Epoch 2: 100%|██████████| 341/341 [02:58<00:00,  1.91it/s, ce=0.2686, cl=0.0279, λ=0.100, lr=0.000085]


...


Epoch 11/20
 - Train CE Loss: 0.5594
 - Train CL Loss: 0.0593
 - Total Loss:    0.5653
 - Learning Rate: 0.00046887


Val Loss: 0.5303 | Val BLEU: 19.70
 -> EarlyStopping counter: 9 out of 10

========== EPOCH 22 ==========


Epoch 12: 100%|██████████| 341/341 [02:58<00:00,  1.91it/s, ce=0.7512, cl=0.0231, λ=0.100, lr=0.000494]



Epoch 12/20
 - Train CE Loss: 0.5977
 - Train CL Loss: 0.0653
 - Total Loss:    0.6043
 - Learning Rate: 0.00049435


Val Loss: 0.5288 | Val BLEU: 20.02
 -> EarlyStopping counter: 10 out of 10
Early stopping triggered at epoch 12
Contrastive Training finished.


In [ ]:
import torch
import gc

# 1. Xóa các biến không còn dùng (nếu bạn biết tên cụ thể)
# Ví dụ: del model, optimizer, loss
# del ten_bien_lon_cua_ban

# 2. Thu gom rác của Python
gc.collect()

# 3. Xóa bộ nhớ cache của CUDA
torch.cuda.empty_cache()

# Kiểm tra lại dung lượng
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   4303 MiB |   9458 MiB |   5336 GiB |   5331 GiB |
|       from large pool |   4048 MiB |   9159 MiB |   5106 GiB |   5102 GiB |
|       from small pool |    254 MiB |    437 MiB |    230 GiB |    229 GiB |
|---------------------------------------------------------------------------|
| Active memory         |   4303 MiB |   9458 MiB |   5336 GiB |   5331 GiB |
|       from large pool |   4048 MiB |   9159 MiB |   5106 GiB |

# Test best model after contrastive + next token prediction training with test set

In [ ]:

# Load the best contrastive model
best_model_path = os.path.join(cl_config.save_dir, "best_cl_model.pt")
print(f"Loading best model from: {best_model_path}")

if not os.path.exists(best_model_path):
    raise FileNotFoundError(f"Best contrastive model not found at {best_model_path}")

best_checkpoint = torch.load(best_model_path, map_location=config.device, weights_only=False)

# Initialize model and projection head
model = TransformerModel(config, vocab_size).to(config.device)
projection = ProjectionHead(config.d_model, cl_config.proj_dim).to(config.device)

# Load state dictionaries
model.load_state_dict(best_checkpoint["model_state_dict"])
projection.load_state_dict(best_checkpoint["projection_state_dict"])

print("Evaluating best model on test set...")
test_loss, test_bleu = beam_evaluate(model, test_loader, criterion, sp, config, beam_size=3)

print(f"\nTest Loss: {test_loss:.4f} | Test BLEU: {test_bleu:.2f}")


Loading best model from: ./checkpoints_contrastive/best_cl_model.pt
Evaluating best model on test set...



Example Pred: Đó không phải hoàn_toàn là những gì tôi muốn .
Example Ref : Nó không phải là thứ tôi muốn xem .

Test Loss: 3.1104 | Test BLEU: 24.28
